In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Asthma"
cohort = "GSE188424"

# Input paths
in_trait_dir = "../../input/GEO/Asthma"
in_cohort_dir = "../../input/GEO/Asthma/GSE188424"

# Output paths
out_data_file = "../../output/preprocess/Asthma/GSE188424.csv"
out_gene_data_file = "../../output/preprocess/Asthma/gene_data/GSE188424.csv"
out_clinical_data_file = "../../output/preprocess/Asthma/clinical_data/GSE188424.csv"
json_path = "../../output/preprocess/Asthma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling of peripheral blood from uncontrolled and controlled asthma"
!Series_summary	"We analyzed the transcriptomes of children with controlled and uncontrolled asthma in Taiwanese Consortium of Childhood Asthma Study (TCCAS). Hierarchical clustering, differentially expressed gene (DEG), weighted gene co-expression network analysis (WGCNA) and pathway enrichment methods were performed, to investigate important genes between two groups."
!Series_overall_design	"Analysis of gene expression obtained from human whole blood comparing uncontrolled and controlled asthma."
Sample Characteristics Dictionary:
{0: ['gender: female', 'gender: male']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Based on the provided information, let's analyze this dataset:

# 1. Gene Expression Data Availability
# The series summary mentions "transcriptomes" and "gene expression profiling"
# which strongly indicates gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Trait (Asthma control status) is mentioned in the background information
# However, we cannot locate it in the sample characteristics dictionary
trait_row = None  # Cannot find in sample characteristics
is_trait_available = False  # Since trait_row is None

# Age data is not available in the sample characteristics
age_row = None

# Gender data is available at key 0
gender_row = 0

# 2.2 Data Type Conversion
# For trait (when we locate it):
def convert_trait(val):
    if val is None:
        return None
    val = val.lower().split(': ')[-1].strip()
    if 'uncontrolled' in val:
        return 1
    elif 'controlled' in val:
        return 0
    return None

# For age (if we found it, which we didn't):
def convert_age(val):
    if val is None:
        return None
    try:
        # Extract the value after the colon and convert to float
        return float(val.split(': ')[-1].strip())
    except:
        return None

# For gender:
def convert_gender(val):
    if val is None:
        return None
    val = val.lower().split(': ')[-1].strip()
    if 'female' in val:
        return 0
    elif 'male' in val:
        return 1
    return None

# 3. Save Metadata
# Trait data is not available in the sample characteristics
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this substep since trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Asthma/GSE188424/GSE188424_series_matrix.txt.gz


Gene data shape: (47235, 99)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers starting with ILMN_ are Illumina probe IDs, not human gene symbols
# These are specific to Illumina microarray platforms and need to be mapped to human gene symbols
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for gene mapping
# From examining the preview, we can see:
# - 'ID' column contains identifiers matching those in the gene expression data (ILMN_*)
# - 'Symbol' column contains gene symbols we want to map to

# 2. Get the gene mapping dataframe by extracting the identifier and symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Mapping dataframe shape: {mapping_df.shape}")
print("First few rows of mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level data to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene-level expression data shape: {gene_data.shape}")
print("First few gene symbols:")
print(gene_data.index[:10])

# 4. Normalize gene symbols to ensure consistency (optional but recommended)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene expression data shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# 5. Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Saved gene expression data to {out_gene_data_file}")


Mapping dataframe shape: (44837, 2)
First few rows of mapping dataframe:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Gene-level expression data shape: (21440, 99)
First few gene symbols:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


Normalized gene expression data shape: (20238, 99)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS'],
      dtype='object', name='Gene')


Saved gene expression data to ../../output/preprocess/Asthma/gene_data/GSE188424.csv


### Step 7: Data Normalization and Linking

In [8]:
# First, re-extract the necessary files from the cohort directory
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get the gene data again 
gene_data = get_genetic_data(matrix_file)

# Read background information and clinical data again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Examine the clinical data structure to see what's actually available
print("Clinical data shape:", clinical_data.shape)
print("Clinical data column names:", clinical_data.columns.tolist())
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics:", sample_characteristics_dict)

# Since we previously determined trait data is not available (trait_row = None),
# we can't create proper clinical data for this dataset
is_trait_available = False

# The gene data has already been normalized and saved in previous steps
print(f"Gene data shape: {gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Since trait data is not available, use is_final=False in validate_and_save_cohort_info
# This bypasses the need for the is_biased parameter
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available
)

print(f"Dataset usability status: {is_usable}")
print("No linked data file saved since trait data is unavailable.")

Clinical data shape: (1, 100)
Clinical data column names: ['!Sample_geo_accession', 'GSM5681954', 'GSM5681955', 'GSM5681956', 'GSM5681957', 'GSM5681958', 'GSM5681959', 'GSM5681960', 'GSM5681961', 'GSM5681962', 'GSM5681963', 'GSM5681964', 'GSM5681965', 'GSM5681966', 'GSM5681967', 'GSM5681968', 'GSM5681969', 'GSM5681970', 'GSM5681971', 'GSM5681972', 'GSM5681973', 'GSM5681974', 'GSM5681975', 'GSM5681976', 'GSM5681977', 'GSM5681978', 'GSM5681979', 'GSM5681980', 'GSM5681981', 'GSM5681982', 'GSM5681983', 'GSM5681984', 'GSM5681985', 'GSM5681986', 'GSM5681987', 'GSM5681988', 'GSM5681989', 'GSM5681990', 'GSM5681991', 'GSM5681992', 'GSM5681993', 'GSM5681994', 'GSM5681995', 'GSM5681996', 'GSM5681997', 'GSM5681998', 'GSM5681999', 'GSM5682000', 'GSM5682001', 'GSM5682002', 'GSM5682003', 'GSM5682004', 'GSM5682005', 'GSM5682006', 'GSM5682007', 'GSM5682008', 'GSM5682009', 'GSM5682010', 'GSM5682011', 'GSM5682012', 'GSM5682013', 'GSM5682014', 'GSM5682015', 'GSM5682016', 'GSM5682017', 'GSM5682018', 'GSM56

Gene data saved to ../../output/preprocess/Asthma/gene_data/GSE188424.csv
Dataset usability status: False
No linked data file saved since trait data is unavailable.
